In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [4]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_routes`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()
rt_sched['month'] =  pd.Series(rt_sched.service_date).dt.month_name()

In [6]:
rt_sched.sample(5)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
1309,300,Big Blue Bus,0,3498,41,2022-06-27,39,33,0.85,Monday,June
325,300,Big Blue Bus,0,3483,5,2022-04-08,27,27,1.00,Friday,April
1375,300,Big Blue Bus,0,3500,43,2022-06-13,35,30,0.86,Monday,June
779,300,Big Blue Bus,0,3490,14,2022-06-25,83,64,0.77,Saturday,June
278,300,Big Blue Bus,0,3483,5,2022-05-09,24,22,0.92,Monday,May


In [7]:
(rt_sched
 >>group_by(_.calitp_itp_id,
            _.agency_name,
            _.calitp_url_number,
            _.service_date,
            _.weekday,
           _.month
           )
 >>summarize(total_num_sched = _.num_sched.sum() ,
             total_num_vp = _.num_vp.sum())
 >>mutate(pct_w_vp = (_.total_num_vp)/(_.total_num_sched))
            )


,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,300,Big Blue Bus,0,2022-04-01,Friday,April,1562,1512,0.97
1,300,Big Blue Bus,0,2022-04-02,Saturday,April,871,635,0.73
2,300,Big Blue Bus,0,2022-04-03,Sunday,April,771,556,0.72
3,300,Big Blue Bus,0,2022-04-04,Monday,April,1518,1087,0.72
4,300,Big Blue Bus,0,2022-04-05,Tuesday,April,1616,1482,0.92
...,...,...,...,...,...,...,...,...,...
86,300,Big Blue Bus,0,2022-06-26,Sunday,June,772,500,0.65
87,300,Big Blue Bus,0,2022-06-27,Monday,June,1465,1017,0.69
88,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1559,1457,0.93
89,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [8]:
# def by_date(df, sum1_sched, sum2_vp):
#     agg_df = (df
#      >>group_by(_.calitp_itp_id,
#                 _.agency_name,
#                 _.calitp_url_number,
#                 _.service_date,
#                 _.weekday,
#                _.month)
#      >>summarize(total_num_sched = (_[sum1_sched].sum()),
#              total_num_vp = (_[sum2_vp].sum()))
#      >>mutate(pct_w_rt = (_.total_num_vp)/(_.total_num_sched))
#             )
#     return agg_df


In [9]:
# cols_to_agg = ['calitp_itp_id', 'calitp_url_number','service_date', 'weekday', 'month']

In [10]:
# rt_sched_agg = by_date(rt_sched, 'num_sched', 'num_vp')

In [11]:
# rt_sched_agg

In [12]:
# check to see if the function in utils works the same

In [13]:
rt_sched_agg = utils.agg_by_date(rt_sched,'num_sched', 'num_vp')

In [14]:
rt_sched_agg

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,300,Big Blue Bus,0,2022-04-01,Friday,April,1562,1512,0.97
1,300,Big Blue Bus,0,2022-04-02,Saturday,April,871,635,0.73
2,300,Big Blue Bus,0,2022-04-03,Sunday,April,771,556,0.72
3,300,Big Blue Bus,0,2022-04-04,Monday,April,1518,1087,0.72
4,300,Big Blue Bus,0,2022-04-05,Tuesday,April,1616,1482,0.92
...,...,...,...,...,...,...,...,...,...
86,300,Big Blue Bus,0,2022-06-26,Sunday,June,772,500,0.65
87,300,Big Blue Bus,0,2022-06-27,Monday,June,1465,1017,0.69
88,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1559,1457,0.93
89,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [15]:
## Looking at Routes

In [16]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

,route_id,n
0,3479,91
1,3480,91
2,3481,91
4,3485,91
5,3486,91
6,3487,91
8,3489,91
9,3490,91
12,3493,91
13,3494,91


In [17]:
rt_sched>>filter(_.service_date=='2022-04-01')>>summarize(n_routes = _.route_id.nunique())

,n_routes
0,19


In [18]:
(rt_sched>>group_by(_.service_date)>>summarize(n_routes=_.route_id.nunique())>>arrange(-_.n_routes)).n_routes.describe()

count   91.00
mean    16.38
std      3.88
min     10.00
25%     11.00
50%     19.00
75%     19.00
max     19.00
Name: n_routes, dtype: float64

In [19]:
#checking one route
(rt_sched>>filter(_.route_id=='3488')
).head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
610,300,Big Blue Bus,0,3488,R10,2022-04-18,6,4,0.67,Monday,April
611,300,Big Blue Bus,0,3488,R10,2022-06-06,6,4,0.67,Monday,June
612,300,Big Blue Bus,0,3488,R10,2022-05-25,6,5,0.83,Wednesday,May
613,300,Big Blue Bus,0,3488,R10,2022-06-07,6,6,1.00,Tuesday,June
614,300,Big Blue Bus,0,3488,R10,2022-04-14,6,6,1.00,Thursday,April


In [20]:
line = alt.Chart(rt_sched).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=('Percent with Vehicle Positions')),
    color='route_short_name',
    strokeDash='route_short_name',
)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_short_name', 'pct_w_vp')
line_chart.properties(width=600)

alt.Chart(...)

In [21]:
rt_sched>>filter(_.pct_w_vp<.5)>>count(_.route_id)

,route_id,n
0,3489,1
1,3491,1
2,3501,55


* something funky happening with route 3501. 
* checking trip updates

In [22]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
1452,300,Big Blue Bus,0,3501,44,2022-04-01,54,51,0.94,Friday,April
1476,300,Big Blue Bus,0,3501,44,2022-04-04,104,0,0.00,Monday,April
1487,300,Big Blue Bus,0,3501,44,2022-04-05,104,0,0.00,Tuesday,April
1470,300,Big Blue Bus,0,3501,44,2022-04-06,104,0,0.00,Wednesday,April
1488,300,Big Blue Bus,0,3501,44,2022-04-07,104,0,0.00,Thursday,April
1446,300,Big Blue Bus,0,3501,44,2022-04-08,52,0,0.00,Friday,April
1483,300,Big Blue Bus,0,3501,44,2022-04-18,104,0,0.00,Monday,April
1459,300,Big Blue Bus,0,3501,44,2022-04-19,104,0,0.00,Tuesday,April
1455,300,Big Blue Bus,0,3501,44,2022-04-20,104,0,0.00,Wednesday,April
1458,300,Big Blue Bus,0,3501,44,2022-04-21,104,0,0.00,Thursday,April


In [23]:

## seeing what the pct_with_rt would look like without route 3501

In [24]:
no_3501 = utils.agg_by_date((rt_sched>>filter(_.route_id != '3501')),'num_sched', 'num_vp')

In [25]:
no_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
10,300,Big Blue Bus,0,2022-04-11,Monday,April,1412,1079,0.76


In [26]:
with_3501 = utils.agg_by_date(rt_sched,'num_sched', 'num_vp')

In [27]:
with_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
6,300,Big Blue Bus,0,2022-04-07,Thursday,April,1628,1505,0.92


In [28]:

line_1 = alt.Chart(with_3501).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    color=alt.value("#136C97"))
line_2 = alt.Chart(no_3501).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    color=alt.value("#E16B26"))
chart = line_1+line_2
line_chart = styleguide.preset_chart_config(chart)
line_chart = _dla_utils.add_tooltip(line_chart, 'service_date:O', 'pct_w_vp:Q')
line_chart.properties(width=600)


alt.LayerChart(...)

In [29]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
463,300,Big Blue Bus,0,3486,8,2022-05-16,71,47,0.66,Monday,May


In [30]:
utils.groupby_onecol(rt_sched, 'month', 'pct_w_vp')

,month,avg
0,April,0.86
2,May,0.85
1,June,0.86


In [31]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
350,300,Big Blue Bus,0,3485,7,2022-04-23,134,95,0.71,Saturday,April


In [33]:
route_weekday = (utils.groupby_twocol(rt_sched, 'route_short_name', 'weekday', 'pct_w_vp', 'weekday'))

In [34]:
route_weekday.sample()

,route_short_name,weekday,avg
65,44,Thursday,0.00


In [37]:
cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [38]:
line = alt.Chart(route_weekday).mark_line().encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'),  sort=cats_day),
    y=alt.Y('avg', title=('Average Percent with Vehicle Positions')),
    color='route_short_name',
    strokeDash='route_short_name',
)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_short_name', 'avg')
line_chart.properties(width=600)

alt.Chart(...)